In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from IPython.display import display, IFrame
from snowflake.snowpark.context import get_active_session
import streamlit as st

session = get_active_session()

st.title("MBI Data Analytics")

In [ ]:
def create_geom(row):
    if 'WKT' in row.keys():
        return wkt.loads(row['WKT'])
    else:
        return Point(row['LONGITUDE'], row['LATITUDE'])

In [ ]:
# Filtering the mbi polygon data to 
q = '''
        select * from MBI__PREMIUM_GEOSPATIAL__MARKET_DATA.PROMOTIONAL."mbi_demographics_jp" 
        where MICROCODE = '13123031007'
    '''

# Execute the query (ensure session is defined and connected to Snowflake)
df = session.sql(q).to_pandas()

# Create geometry column using the helper function
df['GEOM'] = df.apply(create_geom, axis=1)

# Create a GeoDataFrame with the appropriate CRS
gdf = gpd.GeoDataFrame(df, geometry='GEOM', crs="EPSG:4326")


In [ ]:
def get_polygon_coords(geom):
    """
    Extract coordinates from a geometry.
    If it's a Polygon, return its exterior coordinates in a list.
    If it's a MultiPolygon, return a list of exterior coordinates for each polygon.
    """
    if geom.geom_type == 'Polygon':
        return [list(geom.exterior.coords)]
    elif geom.geom_type == 'MultiPolygon':
        return [list(p.exterior.coords) for p in geom.geoms]
    else:
        return None


In [ ]:
import pydeck as pdk
def render_polygon(gdf):
    # Create a DataFrame for the polygon layer by dropping the GEOM column.
    df_poly = pd.DataFrame(gdf.drop(columns=['GEOM']))
    
    # Extract polygon coordinates using the helper function.
    df_poly['coordinates'] = gdf['GEOM'].apply(get_polygon_coords)
    
    # Create the PolygonLayer for pydeck.
    polygon_layer = pdk.Layer(
        'PolygonLayer',
        df_poly,
        get_polygon='coordinates',
        stroked=True,
        extruded=False,
        opacity=0.3,
        filled=True,
        get_fill_color=[0, 255, 0],
        get_line_color=[255, 0, 0],
        pickable=True
    )
    
    # Re-project gdf to a projected CRS (EPSG:3857) for correct centroid calculation.
    gdf_projected = gdf.to_crs(epsg=3857)
    centroids_projected = gdf_projected.centroid
    # Transform the centroids back to the geographic CRS (EPSG:4326).
    centroids = centroids_projected.to_crs(epsg=4326)
    
    # Calculate the mean latitude and longitude from the centroids.
    view_state = pdk.ViewState(
        latitude=centroids.y.mean(),
        longitude=centroids.x.mean(),
        zoom=14,
        bearing=0,
        pitch=0
    )

    # Define a tooltip for extra details on each point.
    tooltip = {
        'html': '''
                <b>Name:</b> {NAME}<br>
                <b>Microcode:</b> {MICROCODE}<br>
                <b>Households: total number:</b> {HH_T}<br>
                <b>Average Household Size:</b> {HH_SIZE}<br>
                <b>Population (males):</b> {MALE}<br>
                <b>Population (Females):</b> {FEMALE}<br>
                <b>Population (University):</b> {EDU_5}<br>
                <b>Purchasing Power: Euro per capita:</b> {PP_EURO} <b>
                
            ''',
        'style': {
            'backgroundColor': 'steelblue',
            'color': 'white'
        }
    }
    
    # Return the pydeck.Deck for rendering.
    return pdk.Deck(
        map_style='mapbox://styles/mapbox/light-v9',
        layers=[polygon_layer],
        initial_view_state=view_state,
        # Uncomment and customize the tooltip if needed:
        tooltip=tooltip
    )


In [ ]:
st.pydeck_chart(render_polygon(gdf))

In [ ]:
q = '''
        select * from MBI__PREMIUM_GEOSPATIAL__MARKET_DATA.PROMOTIONAL."poi_jp" 
        where MICROCODE = '13123031007'
    '''

# Execute the query (assuming 'session' is your active Snowflake session)
df = session.sql(q).to_pandas()

# Create a geometry column using the helper function.
df['GEOM'] = df.apply(create_geom, axis=1)

# Convert LATITUDE and LONGITUDE columns to float.
df['LATITUDE'] = df['LATITUDE'].astype(float)
df['LONGITUDE'] = df['LONGITUDE'].astype(float)

print("DataFrame shape:", df.shape)


In [ ]:
# Create a GeoDataFrame with the appropriate CRS.
gdf = gpd.GeoDataFrame(df, geometry='GEOM', crs="EPSG:4326")
# Optionally, inspect your GeoDataFrame:
#gdf.info()


In [ ]:
def render_points(gdf):
    # Set the initial view using the average of latitude and longitude.
    view_state = pdk.ViewState(
        latitude=gdf['LATITUDE'].mean(),
        longitude=gdf['LONGITUDE'].mean(),
        zoom=14,
        bearing=0,
    )
    
    # Define the ScatterplotLayer.
    layer = pdk.Layer(
        "ScatterplotLayer",
        gdf,
        get_position=["LONGITUDE", "LATITUDE"],
        get_color=[255, 0, 0, 200],
        get_radius=0.5,
        pickable=True,
        opacity=0.8,
        stroked=True,
        filled=True,
        radius_scale=6,
        radius_min_pixels=1,
        radius_max_pixels=100,
        line_width_min_pixels=1,
    )

    # Define a tooltip for extra details on each point.
    tooltip = {
        'html': '''
                <b>PB Id:</b> {PB_ID}<br>
                <b>Name:</b> {NAME}<br>
                <b>Microcode:</b> {MICROCODE}<br>
                <b>Address:</b> {MAIN_ADDRE}<br>
                <b>Post Code:</b> {POSTCODE}<br>
                <b>Main Class:</b> {MAIN_CLASS}<br>
                <b>Bussiness:</b> {BUSINESS_L}<br>
                <b>Group Name:</b> {GROUP_NAME}<br>
                
            ''',
        'style': {
            'backgroundColor': 'steelblue',
            'color': 'white'
        }
    }
    
    # Create and return the pydeck.Deck.
    deck = pdk.Deck(
        map_style='mapbox://styles/mapbox/light-v9',
        layers=[layer],
        initial_view_state=view_state,
        tooltip=tooltip
    )

    return deck

# Render the points on the map.
st.pydeck_chart(render_points(gdf))


In [ ]:
q_poly = '''
        select * from MBI__PREMIUM_GEOSPATIAL__MARKET_DATA.PROMOTIONAL."mbi_demographics_jp" 
        where MICROCODE = '13123031007'
    '''

df_poly = session.sql(q_poly).to_pandas()
# Assume df_poly already has the necessary columns (or a WKT column)
df_poly['GEOM'] = df_poly.apply(create_geom, axis=1)
# Create GeoDataFrame for polygons
gdf_polygons = gpd.GeoDataFrame(df_poly, geometry='GEOM', crs="EPSG:4326")

In [ ]:
q_point = '''
        select * from MBI__PREMIUM_GEOSPATIAL__MARKET_DATA.PROMOTIONAL."poi_jp" 
        where MICROCODE = '13123031007'
    '''

df_point = session.sql(q_point).to_pandas()
df_point['GEOM'] = df_point.apply(create_geom, axis=1)
# Convert LATITUDE and LONGITUDE to float, if needed
df_point['LATITUDE'] = df_point['LATITUDE'].astype(float)
df_point['LONGITUDE'] = df_point['LONGITUDE'].astype(float)
# Create GeoDataFrame for points
gdf_points = gpd.GeoDataFrame(df_point, geometry='GEOM', crs="EPSG:4326")

#print("Polygon DF shape:", gdf_polygons.shape)
#print("Point DF shape:", gdf_points.shape)


In [ ]:
def render_combined_map(gdf_points, gdf_polygons):
    # ----- Polygon Layer -----
    # Prepare a DataFrame for the polygon layer.
    df_poly_layer = pd.DataFrame(gdf_polygons.drop(columns=['GEOM']))
    df_poly_layer['coordinates'] = gdf_polygons['GEOM'].apply(get_polygon_coords)
    
    polygon_layer = pdk.Layer(
        'PolygonLayer',
        df_poly_layer,
        get_polygon='coordinates',
        stroked=True,
        extruded=False,
        opacity=0.3,
        filled=True,
        get_fill_color=[0, 255, 0],
        get_line_color=[255, 0, 0],
        pickable=True
    )
    
    # ----- Point Layer -----
    point_layer = pdk.Layer(
        "ScatterplotLayer",
        gdf_points,
        get_position=["LONGITUDE", "LATITUDE"],
        get_color=[255, 0, 0, 200],
        get_radius=0.5,
        pickable=True,
        opacity=0.8,
        stroked=True,
        filled=True,
        radius_scale=6,
        radius_min_pixels=1,
        radius_max_pixels=100,
        line_width_min_pixels=1,
    )
    
    # ----- Combined View State -----
    # For the point layer, compute average latitude and longitude.
    point_lat = gdf_points['LATITUDE'].mean()
    point_lon = gdf_points['LONGITUDE'].mean()
    
    # For the polygon layer, compute centroids after reprojecting to a projected CRS.
    gdf_poly_proj = gdf_polygons.to_crs(epsg=3857)
    poly_centroids_proj = gdf_poly_proj.centroid
    poly_centroids = poly_centroids_proj.to_crs(epsg=4326)
    poly_lat = poly_centroids.y.mean()
    poly_lon = poly_centroids.x.mean()
    
    # Compute an overall center by averaging the two.
    combined_lat = (point_lat + poly_lat) / 2
    combined_lon = (point_lon + poly_lon) / 2

    view_state = pdk.ViewState(
        latitude=combined_lat,
        longitude=combined_lon,
        zoom=14,
        bearing=0,
        pitch=0
    )

    # Define a tooltip for extra details on each point.
    tooltip = {
        'html': '''
                <b>Name:</b> {NAME}<br>
                <b>Microcode:</b> {MICROCODE}<br>
                <b>Households: total number:</b> {HH_T}<br>
                <b>Average Household Size:</b> {HH_SIZE}<br>
                <b>Population (males):</b> {MALE}<br>
                <b>Population (Females):</b> {FEMALE}<br>
                <b>Population (University):</b> {EDU_5}<br>
                <b>Purchasing Power: Euro per capita:</b> {PP_EURO} <b>
                <b>Address:</b> {MAIN_ADDRE}<br>
                <b>Post Code:</b> {POSTCODE}<br>
                <b>Main Class:</b> {MAIN_CLASS}<br>
                <b>Bussiness:</b> {BUSINESS_L}<br>
                <b>Group Name:</b> {GROUP_NAME}<br>
                
            ''',
        'style': {
            'backgroundColor': 'steelblue',
            'color': 'white'
        }
    }
    
    # ----- Create the Combined Deck -----
    deck = pdk.Deck(
        map_style='mapbox://styles/mapbox/light-v9',
        layers=[polygon_layer, point_layer],
        initial_view_state=view_state,
        tooltip=tooltip
    )
    
    return deck


In [ ]:
st.pydeck_chart(render_combined_map(gdf_points, gdf_polygons))
